In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
import random
random.seed(42)

In [3]:
import numpy as np

In [4]:
import os
os.chdir("../")

In [5]:
import copy

In [6]:
words = open('data/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [7]:
len(words)

32033

In [8]:
## building vocab of chars & mapping to/from integers

chars = sorted(list(set(''.join(words))))
stoi = {s:i + 1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s,i in stoi.items()}

In [9]:
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [10]:
## Building the data set

block_size = 3 ## context length: How many chars to take to predict the next one?

X, Y = [], []

for w in words:
    context = [0] * block_size # -> 1st Iter [0, 0, 0]
    
    for ch in w + ".":

        ix = stoi[ch] ## Encoded values of character
        
        X.append(context) ## 1st Iter [0, 0, 0]
        Y.append(ix) ## 1st Iter [itos[ch]]

        context = context[1:] + [ix] ## Rolling Operation

X = torch.tensor(X)
Y = torch.tensor(Y)

In [11]:
def build_dataset(words: list[str]):
    
    block_size = 3 ## context length: How many chars to take to predict the next one?

    X, Y = [], []

    for w in words:
        context = [0] * block_size # -> 1st Iter [0, 0, 0]

        for ch in w + ".":

            ix = stoi[ch] ## Encoded values of character

            X.append(context) ## 1st Iter [0, 0, 0]
            Y.append(ix) ## 1st Iter [itos[ch]]

            context = context[1:] + [ix] ## Rolling Operation

    X = torch.tensor(X)
    Y = torch.tensor(Y)

    return X, Y
    

In [12]:
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev= build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

#### Understanding Generated X Shape & Y Shape

1. X shape is (N, Context Size). Where N is number of input samples generated while creating X, Y Pairs and Context Size is defined above

2. Y Shape is (N). For each Input Sample, it contains the next character Output

In [13]:
X.shape, X.dtype, 

(torch.Size([228146, 3]), torch.int64)

In [14]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        ...,
        [26, 26, 25],
        [26, 25, 26],
        [25, 26, 24]])

In [15]:
Y.shape, Y.dtype

(torch.Size([228146]), torch.int64)

## Projecting characters into 2 Dimensional Space (Embedding Look Up Table)

Number of Chars: 27

Dimensions: 2

In [16]:
C = torch.randn((27, 2))

### Accessing entries of Look-Up Matrix C

In [17]:
## For 5 encoded char "e"
## itos[5]
C[5]

tensor([ 0.7640, -0.8864])

In [18]:
## For 0 encoded char "a"
## itos[5]
C[0]

tensor([-0.3202,  0.2310])

In [19]:
## Is there a better way to index Look Up Matrix?
## Yes One Hot Encoding
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([ 0.7640, -0.8864])

In [20]:
## First Layer of Neural Language Model is essentially, look up into Embedding matrices for Provide Input

### Getting values for 5th Row & 0th Row simultaneously
C[[5, 0]]

tensor([[ 0.7640, -0.8864],
        [-0.3202,  0.2310]])

In [21]:
### Getting values for 5th Row , 6th Row & 7th Row ,7th Row, 7th Row simultaneously

C[torch.tensor([5, 6, 7, 7, 7])]

tensor([[ 0.7640, -0.8864],
        [-0.5802, -0.8590],
        [ 1.8889, -1.2186],
        [ 1.8889, -1.2186],
        [ 1.8889, -1.2186]])

In [22]:
X.shape, X.dtype, 

(torch.Size([228146, 3]), torch.int64)

### How can I get Embedding Look Up entries for X Input `[N, Context Size]`

In [23]:
## Slicing X Tensor for Single word emma (1st name in data)
X_single_name = X[0:4, :]

In [24]:
X_single_name

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13]])

In [25]:
## In each row of X
## Number of encoded chars == context sizr
## For each encodeed chars -> (, 2) shape Look-Up Embedding Table output

## End Shape would look like
## (N, Context_Size, Embedding Dim)
C[X_single_name], C[X_single_name].shape

(tensor([[[-0.3202,  0.2310],
          [-0.3202,  0.2310],
          [-0.3202,  0.2310]],
 
         [[-0.3202,  0.2310],
          [-0.3202,  0.2310],
          [ 0.7640, -0.8864]],
 
         [[-0.3202,  0.2310],
          [ 0.7640, -0.8864],
          [ 0.3149, -1.0559]],
 
         [[ 0.7640, -0.8864],
          [ 0.3149, -1.0559],
          [ 0.3149, -1.0559]]]),
 torch.Size([4, 3, 2]))

In [26]:
X[13, 2]

tensor(1)

In [27]:
C[X][13,2]

tensor([-0.2002,  0.7638])

In [28]:
C[1]

tensor([-0.2002,  0.7638])

In [29]:
## We can embedd directly by passing X into C and it should work!

emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [30]:
C.shape

torch.Size([27, 2])

In [31]:
X.shape

torch.Size([228146, 3])

In [32]:
X.shape

torch.Size([228146, 3])

In [33]:
### Constructing Hidden Layer

W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [34]:
### Embedding which is input to Hidden Layer has shape of [N, context, embedding dimension] -> (32, 3, 2)
### While Hidden layer neuron shape is (context * embedding dimension, hidden_dim) -> (6, 100)

### Last 2 dimensions of embedding layer needs to combined together so that shape matches for matmul

### One way to solve is to use torch cat function

torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1).shape

torch.Size([228146, 6])

In [35]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1).shape

torch.Size([228146, 6])

In [36]:
### More generic way would be use torch unbind and then use torch cat
### Unbinding is done on dimension where context size is defined

torch.cat(torch.unbind(emb, dim=1), dim=1)

tensor([[-0.3202,  0.2310, -0.3202,  0.2310, -0.3202,  0.2310],
        [-0.3202,  0.2310, -0.3202,  0.2310,  0.7640, -0.8864],
        [-0.3202,  0.2310,  0.7640, -0.8864,  0.3149, -1.0559],
        ...,
        [ 0.3826,  0.4228,  0.3826,  0.4228,  0.2213, -1.6582],
        [ 0.3826,  0.4228,  0.2213, -1.6582,  0.3826,  0.4228],
        [ 0.2213, -1.6582,  0.3826,  0.4228,  0.5852, -1.4571]])

In [37]:
Input_Embedding_Proj = copy.deepcopy(C[X])

In [38]:
Input_Embedding_Proj.shape

torch.Size([228146, 3, 2])

In [39]:
#### Easier way to do this

a = torch.arange(18)
a 

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [40]:
a.view(2, 9)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17]])

In [41]:
emb.shape

torch.Size([228146, 3, 2])

In [42]:
h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1)

In [43]:
h.shape

torch.Size([228146, 100])

In [44]:
### Output Layer

W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [45]:
logits = h @ W2 + b2

In [46]:
logits.shape

torch.Size([228146, 27])

In [47]:
Y.shape

torch.Size([228146])

In [48]:
loss = F.cross_entropy(logits, Y)

In [49]:
h.shape

torch.Size([228146, 100])

In [50]:
logits.shape

torch.Size([228146, 27])

#### Training Loop 

In [51]:
X.shape, Y.shape ## DataSet

(torch.Size([228146, 3]), torch.Size([228146]))

In [52]:
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((27, 2), generator=g, requires_grad=True)

W1 = torch.randn((6, 100), generator=g, requires_grad=True)
b1 = torch.randn(100, generator=g, requires_grad=True)

W2 = torch.randn((100, 27), generator=g, requires_grad=True)
b2 = torch.randn(27, generator=g, requires_grad=True)

parameters = [C, W1, b1, W2, b2]

In [53]:
sum(p.nelement() for p in parameters)

3481

In [49]:
## Estimating Optimal Learning Rate

### Why we want to do that?
### so that we we get optimal learning rate, leading to reduction in loss in less amount of time & computation

### How to do that?

#### Compute loss for different learning rates & at different epochs
#### start with low learning rate, then gradually increase it
#### Select LR which gives lower loss



In [50]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10 ** lre

In [51]:
lri = []

lossi = []

for i in range(1000):

    # Adding MiniBatch Gradient Descent
    ix = torch.randint(0, X.shape[0], (32,))
    
    # Forward Pass
    emb = C[X[ix]]

    h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1)

    logits = h @ W2 + b2

    loss = F.cross_entropy(logits, Y[ix])


    # print(loss.item())

    ## Backward pass 

    for p in parameters:
        p.grad = None

    loss.backward()

    ## Update

    ## Trying to find optimal learning rate
    lr = lrs[i]
    for p in parameters: 
        p.data += -lr * p.grad

    lri.append(lre[i].item())
    lossi.append(loss.item())

# print(loss.item())


In [ ]:
lri

In [ ]:
plt.plot(lri, lossi)

In [49]:
g = torch.Generator().manual_seed(217483647)

block_size = 3

C = torch.randn((27, 10), generator=g, requires_grad=True)

W1 = torch.randn((block_size * C.shape[-1], 200), generator=g, requires_grad=True)
b1 = torch.randn(200, generator=g, requires_grad=True)

W2 = torch.randn((200, 27), generator=g, requires_grad=True)
b2 = torch.randn(27, generator=g, requires_grad=True)

parameters = [C, W1, b1, W2, b2]

In [ ]:
C.shape[-1]

In [ ]:
sum(p.nelement() for p in parameters)

In [52]:
lri = []
lossi = []
stepi = []

In [ ]:
for i in range(100000):

    # Adding MiniBatch Gradient Descent
    ix = torch.randint(0, Xtr.shape[0], (32,))
    
    # Forward Pass
    emb = C[Xtr[ix]]

    h = torch.tanh(emb.view(emb.shape[0], block_size * C.shape[-1]) @ W1 + b1)

    logits = h @ W2 + b2

    loss = F.cross_entropy(logits, Ytr[ix])


    # print(loss.item())

    ## Backward pass 

    for p in parameters:
        p.grad = None

    loss.backward()

    ## Update

    ## Trying to find optimal learning rate
    lr = 0.1 / 35
    for p in parameters: 
        p.data += -lr * p.grad
        
    stepi.append(i)
    lossi.append(loss.log10().item())

print(loss)


In [ ]:
Ytr.shape

In [ ]:
h.shape

In [ ]:
loss

In [ ]:
plt.plot(stepi, lossi)

### Train

In [ ]:
emb = C[Xtr]

h = torch.tanh(emb.view(emb.shape[0], block_size * C.shape[-1]) @ W1 + b1)

logits = h @ W2 + b2

loss = F.cross_entropy(logits, Ytr)

loss

### Validation

In [ ]:
emb = C[Xdev]

h = torch.tanh(emb.view(emb.shape[0], block_size * C.shape[-1]) @ W1 + b1)

logits = h @ W2 + b2

loss = F.cross_entropy(logits, Ydev)

loss

## TEST

In [ ]:
# emb = C[Xte]

# h = torch.tanh(emb.view(emb.shape[0], block_size * C.shape[-1]) @ W1 + b1)

# logits = h @ W2 + b2

# loss = F.cross_entropy(logits, Yte)

# loss

In [ ]:
## Training Split, Validation Split, Test Split 
# 80%, 10%, 10%

### Visualizing Character embedding matrix

In [ ]:
plt.figure(figsize=(8, 8))

plt.scatter(C[:, 0].data, C[:, 1].data, s=200)

for i in range(C.shape[0]):
    plt.text(C[i,0].item(), C[i, 1].item(), itos[i], ha="center", va="center", color="white")

plt.grid('minor')

In [104]:
import numpy as np
from sklearn.manifold import TSNE, SpectralEmbedding

In [107]:
tsne_embed = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=5)

se_ = SpectralEmbedding(n_components=2)


lower_tsne_proj = tsne_embed.fit_transform(C.data.numpy())
# lower_tsne_proj = se_.fit_transform(C.data.numpy())

In [ ]:
plt.figure(figsize=(8, 8))

plt.scatter(lower_tsne_proj[:, 0], lower_tsne_proj[:, 1], s=200)

for i in range(C.shape[0]):
    plt.text(lower_tsne_proj[i,0], lower_tsne_proj[i, 1], itos[i], ha="center", va="center", color="white")

plt.grid('minor')

### Inferencing

In [ ]:
emb = C[Xdev]

h = torch.tanh(emb.view(emb.shape[0], block_size * C.shape[-1]) @ W1 + b1)

logits = h @ W2 + b2

loss = F.cross_entropy(logits, Ydev)

loss

In [ ]:
W1.shape

In [ ]:
for k in range(35):

    initial_context = [0 , 0 , 0]
    out = []

    while True:

        emb = C[initial_context] ## Embedding Look Up

        h = torch.tanh(emb.view(1, block_size * C.shape[-1]) @ W1 + b1) ## Hidden Layer Output

        logits_ = h @ W2 + b2

        prob_ = logits_.softmax(dim=-1)

        
        ix = torch.multinomial(prob_, num_samples=1, replacement=True, generator=g).item()

        initial_context.append(ix)

        initial_context = initial_context[1:]

        if ix == 0:
            break

        out.append(itos[ix])



    
    print("".join(out))